**Author name:** Thanh Chung Nguyen



In [ ]:
# !pip install plotly

In [8]:
# import statements
from time import sleep
from kafka import KafkaConsumer
import datetime as dt
import time
import json
import plotly.express as px
import re, json, requests
import pandas as pd
from IPython.display import clear_output
import ast
import warnings
warnings.filterwarnings("ignore")

In [9]:
# read data 
url = 'https://raw.githubusercontent.com/nikhilkumarsingh/choropleth-python-tutorial/master/states_india.geojson'
resp = requests.get(url)
data = json.loads(resp.text)

# convert attributes name of state to uppercase
for i in range(len(data['features'])):
    data['features'][i]['properties']['st_nm'] = data['features'][i]['properties']['st_nm'].upper()

# get state and id of map
state_id_map = {}
for feature in data["features"]:
    feature["id"] = feature["properties"]["state_code"]
    state_id_map[feature["properties"]["st_nm"]] = feature["id"]

In [12]:
def connect_kafka_consumer():
    _consumer = None
    try:
         _consumer = KafkaConsumer("final_best",
                                   consumer_timeout_ms=10000, # stop iteration if no message after 10 sec
                                   auto_offset_reset='earliest', # comment this if you don't want to consume earliest available message
                                   bootstrap_servers=['localhost:9092'],
                                   api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka')
        print(str(ex))
    finally:
        return _consumer

In [ ]:

def consume_messages(consumer):
    try:
        # loop each message of consumer
        for message in consumer:
            # clear output of graph each iteration
            clear_output(wait=True)
            
            # transform message to data
            nest_value = str(message.value.decode('utf-8'))
            nest_value = ast.literal_eval(nest_value)
            
            # convert list of dictionary to dataframe
            df = pd.DataFrame(nest_value)

            # preparation of variable to update dataframe
            count = 0
            mapping_state = df["State"].tolist()
            count = df.shape[0]
            
            # update final dataframe to draw graph
            for i in state_id_map:
                if i not in mapping_state:
                    df['State'][count] = i
                    df['count'][count] = 0
                    
                    count+= 1
                    

            # draw the graph
            fig = px.choropleth(df, geojson=data, color = "count",
                                locations="State", featureidkey="properties.st_nm",color_continuous_scale='blues',
                               )
            fig.update_geos(fitbounds="locations", visible=False)
            fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
            
            fig.show()
            
            time.sleep(0.5)
            
            
            
                    
    except Exception as ex:
        print(str(ex))
        
    
if __name__ == '__main__':
    consumer = connect_kafka_consumer()
    consume_messages(consumer)

: 